# Standard lineshapes

```{note}
The formulas and figures on this page have been generated with the lineshapes in the {mod}`.lineshape` module, so as to [glue](https://myst-nb.readthedocs.io/en/latest/use/glue.html) them back in to the API of that module.
```

In [ ]:
import myst_nb
import sympy as sp
from expertsystem.amplitude.dynamics.lineshape import (
    BlattWeisskopf,
    relativistic_breit_wigner,
    relativistic_breit_wigner_with_ff,
)

In [ ]:
import matplotlib.pyplot as plt
from expertsystem.reaction.combinatorics import arange
from matplotlib.figure import Figure


def plot_real_imag(
    expression: sp.Expr,
    variable: sp.Symbol,
    x_min: float,
    x_max: float,
    resolution: int = 100,
) -> Figure:
    delta = (x_max - x_min) / resolution
    x = list(arange(x_min, x_max, delta))
    y_real = list(map(lambda x: sp.Abs(expression).subs(variable, x), x))
    y_imag = list(map(lambda x: sp.arg(expression).subs(variable, x), x))
    fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8, 8))
    for a in ax:
        a.xaxis.set_ticks([])
        a.yaxis.set_ticks([])
    ax_real, ax_imag = ax
    ax_imag.set(xlabel=f"${variable.name}$")
    ax_imag.set(ylabel=f"imag $f({variable.name})$")
    ax_real.set(ylabel=f"real $f({variable.name})$")
    ax_real.yaxis.set_ticks([])
    ax_imag.yaxis.set_ticks([0, float(sp.pi)])
    ax_imag.yaxis.set_ticklabels([0, R"$\pi$"])
    ax_imag.set_ylim([0, float(sp.pi)])
    ax_real.plot(x, y_real)
    ax_imag.plot(x, y_imag)
    return fig

## Form factor

The {mod}`expertsystem` uses {class}`.BlattWeisskopf` functions $B_L$ as _barrier factors_ (also called _form factors_):

In [ ]:
q, d, L = sp.symbols("q, d, L", real=True)
ff = BlattWeisskopf(q, d, L)
display(ff)
myst_nb.glue("BlattWeisskopf", ff.doit())

BlattWeisskopf(q, d, L)

Piecewise((1, Eq(L, 0)), (2*d**2*q**2/(d**2*q**2 + 1), Eq(L, 1)), (13*d**4*q**4/(9*d**2*q**2 + (d**2*q**2 - 3)**2), Eq(L, 2)), (277*d**6*q**6/(d**2*q**2*(d**2*q**2 - 15)**2 + (2*d**2*q**2 - 5)*(18*d**2*q**2 - 45)), Eq(L, 3)), (12746*d**8*q**8/(25*d**2*q**2*(2*d**2*q**2 - 21)**2 + (d**4*q**4 - 45*d**2*q**2 + 105)**2), Eq(L, 4)))

## Relativistic Breit-Wigner

### _Without_ form factor

See {func}`.relativistic_breit_wigner`:

In [ ]:
m, m0, w0 = sp.symbols("m m0 Gamma", real=True)
myst_nb.glue(
    "relativistic_breit_wigner",
    relativistic_breit_wigner(m, m0, w0),
)

Gamma*m0/(-I*Gamma*m0 - m**2 + m0**2)

In [ ]:
plot_real_imag(relativistic_breit_wigner(m, 1.0, 0.3), m, x_min=0, x_max=2);

### _With_ form factor

See {func}`.relativistic_breit_wigner_with_ff`:

In [ ]:
L = 0
m, m0, w0, ma, mb, meson_radius = sp.symbols(
    "m m0 Gamma m_a m_b q_r", real=True
)
myst_nb.glue(
    "relativistic_breit_wigner_with_ff",
    relativistic_breit_wigner_with_ff(
        mass=m,
        mass0=m0,
        gamma0=w0,
        m_a=ma,
        m_b=mb,
        angular_momentum=L,
        meson_radius=meson_radius,
    ).doit(),
)

Gamma*m0**2*sqrt((m**2 - (m_a - m_b)**2)*(m**2 - (m_a + m_b)**2)/m**2)/(m*sqrt((m0**2 - (m_a - m_b)**2)*(m0**2 - (m_a + m_b)**2)/m0**2)*(-I*Gamma*m0**2*sqrt((m**2 - (m_a - m_b)**2)*(m**2 - (m_a + m_b)**2)/m**2)/(m*sqrt((m0**2 - (m_a - m_b)**2)*(m0**2 - (m_a + m_b)**2)/m0**2)) - m**2 + m0**2))

In [ ]:
ma = 0.2
mb = 0.3
complex_bw_ff = relativistic_breit_wigner_with_ff(
    mass=m,
    mass0=1.0,
    gamma0=0.3,
    m_a=ma,
    m_b=mb,
    angular_momentum=0,
    meson_radius=1,
)
plot_real_imag(complex_bw_ff.doit(), m, x_min=ma + mb, x_max=2);